# Newton's basins

Newton's basins is a family of fractals that shows how found root of equation depends on selected starting point using Newton's method.
The Newton's method is a way to find an approximate root of an equation using the first derivative of a equation function.
The basins can be kinda beautiful, so I want to create a way to draw them.

Firstly let's get some config values and constants.

In [1]:
target_roots = [
    1.2 - 4*1j,
    3 + 2.4*1j,
    -1.2 - 1.4*1j
]
max_iterations = 100

# screen_x_max = 2560
# screen_y_max = 1440
screen_x_max = 256
screen_y_max = 256
eye_vx = 2. - 0.2
eye_vy = 1. - 0.4
eye_zoom = 40.

screen_size = (screen_x_max, screen_y_max)
vx0, vy0 = screen_x_max / 2., screen_y_max / 2.
zoom = min(screen_x_max, screen_y_max) * eye_zoom / 10.

class DataFields:
    screen_x = 'Screen X'
    screen_y = 'Screen Y'
    root_virtual_x = 'Root Virtual X'
    root_virtual_y = 'Root Virtual Y'
    converged = 'Converged'
    label = 'Label'
    color = 'Color'

v_min = 80
v_max = 100
saturation_value = 90
color_by_labels = {
    # hsv - (h, s, v_min, v_max)
    0: (00, saturation_value, v_min, v_min),
    1: (40, saturation_value, v_min, v_min),
    2: (70, saturation_value, v_min, v_min),
}

Then let's use these constants to define coordinates convertions function (screen to world and vice versa).

In [2]:
def virtual_to_screen(vx: float, vy: float) -> (int, int):
    sx = int(round(vx0 - (eye_vx - vx) * zoom))
    sy = int(round(vy0 - (vy - eye_vy) * zoom))
    return sx, sy

def screen_to_virtual(sx: int, sy: int) -> (float, float):
    vx = (sx - vx0) / zoom + eye_vx
    vy = (vy0 - sy) / zoom + eye_vy
    return vx, vy

In the configuration we have a list of roots. To create basins we neet to create a function out of it and then find the first and the second derivatives for this function.

Code bellow uses symbolic calculations to define that function as multiplication of `(x - root)` and then find the first derivative.
The function and the derivatives are built from expressions.

In [3]:
from sympy import symbols, diff, lambdify
from math import prod

def create_function(roots):
    variable = symbols('z')
    func_f = prod(map(lambda root: variable - root, roots))
    func_df = diff(func_f, variable)
    return lambdify(variable, func_f, 'numpy'), lambdify(variable, func_df, 'numpy')

f, df = create_function(target_roots)

Now that we have the function and the derivative we can use the Newton method of approximate equations solving to create basins.

To do that we convert every point of screen (image) into world (virtual) Decart's coordinates and acquired coordinates as seed points (`x + yi`) to find a some of the roots of the equation.

In [4]:
import numpy as np
import pandas as pd
import scipy.optimize as sopt

def get_roots_out_of_screen_points():
    sps = np.array([(sx, sy) for sx in range(screen_size[0]) for sy in range(screen_size[1])])
    spsx, spsy = sps[:, 0], sps[:, 1]
    vpsx, vpsy = screen_to_virtual(spsx, spsy)
    vps = vpsx + vpsy * 1j
    res = sopt.newton(f, fprime=df, x0=vps, maxiter=max_iterations, full_output=True)
    return pd.DataFrame.from_dict({
        DataFields.screen_x: spsx,
        DataFields.screen_y: spsy,
        DataFields.root_virtual_x: np.vectorize(lambda z: z.real)(res.root),
        DataFields.root_virtual_y: np.vectorize(lambda z: z.imag)(res.root),
        DataFields.converged: res.converged,
    })

approximate_roots = get_roots_out_of_screen_points()

At this point we have a table of every screen (image) point with their approximate root coordinates and number of iterations that it took to find a root.
Now we can use K-means method to clusterize every screen point by the root it leads to.
This info will be used to paint every starting point to color that represents every root.

In [5]:
from sklearn.cluster import KMeans

model = KMeans(n_clusters=len(target_roots), n_init=8, random_state=42)
clusters = model.fit_predict(approximate_roots[[DataFields.root_virtual_x, DataFields.root_virtual_y]].values)
approximate_roots[DataFields.label] = pd.DataFrame(clusters)

Now we can add color for every point in the table.

The trick is that every root have its basic color. Every point that leads to that root (via newton method) use that basic color but value changes based of number of iterations it took to find out the root.

That trick helps us get more beautiful image with some contrast.

In [6]:
from colorsys import hsv_to_rgb

def get_color_by_cluster_and_iterations(root_row: pd.DataFrame) -> (int, int, int):
    label = np.int8(root_row[DataFields.label])
    hue, saturation, value_min, value_max = color_by_labels[label]
    converged = root_row[DataFields.converged]
    if converged:
        hue, saturation, value = hue / 100., saturation / 100., value_max / 100.
        r, g, b = hsv_to_rgb(hue, saturation, value)
        return int(r * 255), int(g * 255), int(b * 255)
    else:
        return 0, 0, 0

approximate_roots[DataFields.color] = approximate_roots.apply(get_color_by_cluster_and_iterations, axis=1)

We have all we need to print basins. To do that we have to iterate over all rows in our table and paint described points to specified colors.

In [7]:
from PIL import Image

img = Image.new("RGB", screen_size, "black")

def is_in_image(x, y):
    return (0 <= x < screen_size[0] - 1) and (0 <= y < screen_size[1] - 1)

def generate_basins_image(image: Image):
    pixels = image.load()
    for _, row in approximate_roots.iterrows():
        sx = row[DataFields.screen_x]
        sy = row[DataFields.screen_y]
        pixels[sx, sy] = row[DataFields.color]
    for root in target_roots:
        vx, vy = root.real, root.imag
        sx, sy = virtual_to_screen(vx, vy)
        print(vx, vy, sx, sy)
        if is_in_image(sx, sy):
            pixels[sx, sy] = (255, 255, 255)
    svx0, svy0 = virtual_to_screen(0, 0)
    if is_in_image(svx0, svy0):
        pixels[svx0, svy0] = (0, 0, 0)

generate_basins_image(img)

1.2 -4.0 -486 4838
3.0 2.4 1357 -1715
-1.2 -1.4 -2944 2176


It's all done now. Time to show the result!

In [8]:
img.show()
img.save('basins.png')